In [ ]:
!pip install timm

In [ ]:
import torchvision
import torch
import pickle

In [ ]:
import timm

In [ ]:
data = torchvision.datasets.Places365(root='./datasets', split='train-standard', small=True, download=True)

100%|██████████| 67498496/67498496 [00:01<00:00, 62367841.48it/s]


Extracting ./datasets/filelist_places365-standard.tar to ./datasets


100%|██████████| 26103685120/26103685120 [05:35<00:00, 77710003.78it/s]


Extracting ./datasets/train_256_places365standard.tar to ./datasets


In [ ]:
from torchvision.datasets import Places365

In [ ]:
from fastai.vision.all import *

In [ ]:
def label_func(fname):
    if len(Path(fname).parent.name) > 2:
        return Path(fname).parent.name + Path(fname).parent.parent.name
    else:
        return Path(fname).parent.name

In [ ]:
data_prop = DataBlock(
    blocks = (ImageBlock,CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.2, seed=42),
    get_y = label_func,
    item_tfms = RandomResizedCrop(256, min_scale=0.9),
    batch_tfms = aug_transforms()
)

In [ ]:
data_prop2 = DataBlock(
    blocks = (ImageBlock,CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.2, seed=42),
    get_y = label_func,
    item_tfms = Resize((224, 224)),
    batch_tfms = aug_transforms()
)

In [ ]:
dls  = data_prop.dataloaders('./datasets/data_256_standard/', bs=256)

In [ ]:
dls2 = data_prop2.dataloaders('./datasets/data_256_standard/', bs=224)

In [ ]:
learner1 =vision_learner(dls, 'resnet50d.ra2_in1k' ,metrics=[error_rate, accuracy, top_k_accuracy])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
learner2 =vision_learner(dls2, 'coatnet_nano_rw_224.sw_in1k' ,metrics=[error_rate, accuracy, top_k_accuracy])

In [ ]:
learner1 = learner1.load('Myresnet50d')

/usr/local/lib/python3.10/dist-packages/fastai/learner.py:59: UserWarning: Saved file doesn't contain an optimizer state.
  elif with_opt: warn("Saved file doesn't contain an optimizer state.")


In [ ]:
#import pathlib
#plt = platform.system()
#if plt == 'Windows': pathlib.PosixPath = pathlib.WindowsPath

In [ ]:
learner2 = load_learner('coatnet.pkl')

In [ ]:
model1= learner1.model

In [ ]:
model2= learner2.model

In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
if torch.cuda.is_available():
    device =  torch.device('cuda')
else:
    device =  torch.device('cpu')

In [ ]:

OUR_CLASSES=365

In [ ]:
class HybridN(torch.nn.Module):
    def __init__(self, model1, model2) -> None:
        super(HybridN, self).__init__()
        self.mdl1 = model1
        self.mdl2 = model2
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(in_features=OUR_CLASSES*2, out_features=512),
            torch.nn.ReLU(),
            torch.nn.Linear(512,365)
        )

    def forward(self, x):
        self.mdl1.eval()
        self.mdl2.eval()
        out1 = self.mdl1(x)
        out2 = self.mdl2(x)
        out = torch.cat((out1, out2), dim=1)
        x = self.fc(out)
        return x


In [ ]:
to_device(model1, device)
to_device(model2, device)
big = HybridN(model1, model2)
to_device(big, device)

HybridN(
  (mdl1): Sequential(
    (0): TimmBody(
      (model): ResNet(
        (conv1): Sequential(
          (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU(inplace=True)
          (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), 

In [ ]:
root_dir = './datasets'

In [ ]:
from torchvision import datasets, transforms

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
                                transforms.Resize(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ColorJitter(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])


test_transform = transforms.Compose([
                                transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])





train_data = Places365(root=root_dir, split='train-standard', small=True, download=False, transform=train_transform)
test_data = Places365(root=root_dir, split='val', small=True, download=False, transform=test_transform)

In [ ]:
train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False, num_workers=4)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.Adam(big.fc.parameters(), lr=0.001)

In [ ]:
if torch.cuda.is_available():
    device =  torch.device('cuda')
else:
    device =  torch.device('cpu')

In [ ]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.
    correct = 0
    total = 0

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_loader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = big(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()




        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            running_loss = 0.



    accuracy = correct / total
    return last_loss, accuracy

In [ ]:
import datetime

In [ ]:
torch.cuda.empty_cache()

In [ ]:
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    big.train(True)
    avg_loss = train_one_epoch(epoch_number)


    running_vloss = 0.0
    correct = 0
    total = 0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    big.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(test_loader):
            vinputs, vlabels = vdata
            vinputs, vlabels = vinputs.cuda(), vlabels.cuda()
            voutputs = big(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss


            # Calculate validation accuracy
            _, vpredicted = torch.max(voutputs.data, 1)
            total += vlabels.size(0)
            correct += (vpredicted == vlabels).sum().item()


    avg_vloss = running_vloss / (i + 1)
    val_accuracy = correct / total

    print(f'LOSS train {avg_loss} valid {avg_vloss}')
    print(f'valid {val_accuracy}')



    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}'.format(epoch_number)
        torch.save(big.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
  batch 1000 loss: 2.3296201545000077
  batch 2000 loss: 2.0179916833639147
  batch 3000 loss: 1.9680192667245864
  batch 4000 loss: 1.9675359259843825
  batch 5000 loss: 1.9446904710531234
  batch 6000 loss: 1.9231701859235764
  batch 7000 loss: 1.9307630525827408
  batch 8000 loss: 1.9204145077466965
  batch 9000 loss: 1.9117811543941499
  batch 10000 loss: 1.9058752139806747
  batch 11000 loss: 1.9041150370836257
  batch 12000 loss: 1.9067993041276932
  batch 13000 loss: 1.8946286444664002
  batch 14000 loss: 1.885815178990364
LOSS train (1.885815178990364, 0.4790014749426103) valid 1.950008749961853
valid 0.48041095890410956
EPOCH 2:
  batch 1000 loss: 1.8867945058345794
  batch 2000 loss: 1.8931945152282714
  batch 3000 loss: 1.8849522304534911
  batch 4000 loss: 1.890642570257187
  batch 5000 loss: 1.8807494660615922
  batch 6000 loss: 1.8810622565746307
  batch 7000 loss: 1.8811723428964615
  batch 8000 loss: 1.877561089515686
  batch 9000 loss: 1.8786720273494721
  bat

In [ ]:
model_path = 'hybrid_model.pth'
torch.save(big.state_dict(), model_path)